In [1]:
!nvidia-smi

Fri Jun 17 07:27:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [1]:
%cd /content/drive/MyDrive/vessel_segmentation/

/content/drive/MyDrive/vessel_segmentation


In [4]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 9.6 MB/s 


In [ ]:
!pip install visualkeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 992 kB 7.6 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

K.set_image_data_format('channels_last')

def jaccard(y_true, y_pred):
  smooth = 1.
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  y_pred_f = tf.cast(y_pred_f, tf.float32)
  y_true_f = tf.cast(y_true_f, tf.float32)
  intersection = K.sum( y_true_f * y_pred_f)
  union = K.sum(y_true_f + y_pred_f - y_true_f * y_pred_f)
  return (intersection + smooth) / (union + smooth)

def dice_coef(y_true, y_pred):
  smooth = 1.
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  y_pred_f = tf.cast(y_pred_f, tf.float32)
  y_true_f = tf.cast(y_true_f, tf.float32)
  intersection = K.sum(y_true_f * y_pred_f)
  return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def customized_loss(y_true, y_pred):
  return tf.math.log(tf.math.cosh(1-dice_coef(y_true,y_pred)))

## MultiResUNet

```python
def step_decay(epoch):
   initial_lrate = LRATE
   drop = 0.5
   epochs_drop = 10.0 # drop lr by half every 10 epochs
   lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
   return lrate

lr_scheduler = LearningRateScheduler(step_decay)
```

- 12 train
- 8 validation

In [ ]:
from model.OurMultiResUNet import *
from keras.callbacks import ModelCheckpoint, TensorBoard
from generator import *
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow_addons as tfa
# import visualkeras

IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 1

model = OurMultiResUNet(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
model.summary()

# visualkeras.layered_view(model, legend=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 512, 512, 8)  72          ['input_1[0][0]']                
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 512, 512, 8)  24         ['conv2d_1[0][0]']               
 rmalization)                                                                                     
                                                                                              

In [ ]:
config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
sess = tf.compat.v1.Session(config=config)

EPOCHS = 100
LOSS = customized_loss

train_dataset = './data/artery/train_artery.h5'
valid_dataset = './data/artery/valid_artery.h5'
BATCH_SIZE = 4

train_gen = DatasetGenerator(dataset_path=train_dataset,batch_size=BATCH_SIZE)
TOTAL = train_gen.getNumberOfTotalImages()
train_epoch = train_gen.generator()

valid_gen = DatasetGenerator(dataset_path=valid_dataset,batch_size=BATCH_SIZE)
TOTAL_VAL = valid_gen.getNumberOfTotalImages()
valid_epoch = valid_gen.generator()

steps_per_epoch = int(TOTAL / BATCH_SIZE)

loss_func = LOSS
    
steps_per_epoch = int(TOTAL / BATCH_SIZE)
cycles_n = 30
step_size = EPOCHS * steps_per_epoch / (cycles_n*2)
LRATE = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=1e-4,
                                            maximal_learning_rate=1e-3,
                                            step_size=step_size,
                                            scale_fn=lambda x: 1.0)

optimizer_func = Adam(learning_rate=LRATE)

model.compile(optimizer=optimizer_func, loss=loss_func, metrics=[dice_coef, jaccard, 'accuracy'])

model_checkpoint = ModelCheckpoint(
    filepath='./checkpoints/model_{val_dice_coef:.4f}.h5',
    monitor='val_dice_coef',
    verbose=1,
    save_best_only=True,
    mode='max')

callbacks = [model_checkpoint]

history = model.fit(train_epoch,
          steps_per_epoch=int(TOTAL / BATCH_SIZE),
          verbose=1,
          epochs=EPOCHS,
          shuffle=True,
          validation_data=valid_epoch,
          validation_steps=int(TOTAL_VAL / BATCH_SIZE),
          callbacks=callbacks)

train_gen.close()
valid_gen.close()

The number of total images: 1660
The number of total images: 433
Epoch 1/100
415/415 [==============================] - ETA: 0s - loss: 0.4235 - dice_coef: 0.0135 - jaccard: 0.0068 - accuracy: 0.8361
Epoch 1: val_dice_coef improved from -inf to 0.01188, saving model to ./checkpoints/model_0.0119.h5
415/415 [==============================] - 343s 767ms/step - loss: 0.4235 - dice_coef: 0.0135 - jaccard: 0.0068 - accuracy: 0.8361 - val_loss: 0.4248 - val_dice_coef: 0.0119 - val_jaccard: 0.0060 - val_accuracy: 0.8412
Epoch 2/100
415/415 [==============================] - ETA: 0s - loss: 0.4214 - dice_coef: 0.0164 - jaccard: 0.0083 - accuracy: 0.9347
Epoch 2: val_dice_coef improved from 0.01188 to 0.01625, saving model to ./checkpoints/model_0.0162.h5
415/415 [==============================] - 293s 706ms/step - loss: 0.4214 - dice_coef: 0.0164 - jaccard: 0.0083 - accuracy: 0.9347 - val_loss: 0.4215 - val_dice_coef: 0.0162 - val_jaccard: 0.0082 - val_accuracy: 0.9365
Epoch 3/100
415/415 [===

In [ ]:
from model.OurMultiResUNet import *
from keras.callbacks import ModelCheckpoint, TensorBoard
from generator import *
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow_addons as tfa
from keras.models import load_model

config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
sess = tf.compat.v1.Session(config=config)

IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 1

EPOCHS = 20
LOSS = customized_loss

train_dataset = './data/artery/train_artery.h5'
valid_dataset = './data/artery/valid_artery.h5'
BATCH_SIZE = 4

train_gen = DatasetGenerator(dataset_path=train_dataset,batch_size=BATCH_SIZE)
TOTAL = train_gen.getNumberOfTotalImages()
train_epoch = train_gen.generator()

valid_gen = DatasetGenerator(dataset_path=valid_dataset,batch_size=BATCH_SIZE)
TOTAL_VAL = valid_gen.getNumberOfTotalImages()
valid_epoch = valid_gen.generator()

steps_per_epoch = int(TOTAL / BATCH_SIZE)

model = load_model("./checkpoints/model_0.7909.h5",compile=False)

loss_func = LOSS
    
steps_per_epoch = int(TOTAL / BATCH_SIZE)
cycles_n = 30
step_size = EPOCHS * steps_per_epoch / (cycles_n*2)
LRATE = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=5e-4,
                                            maximal_learning_rate=5e-3,
                                            step_size=step_size,
                                            scale_fn=lambda x: 1.0)

optimizer_func = SGD(learning_rate=LRATE,momentum=0.97)

model.compile(optimizer=optimizer_func, loss=loss_func, metrics=[dice_coef, jaccard, 'accuracy'])

model_checkpoint = ModelCheckpoint(
    filepath='./checkpoints/model_{val_dice_coef:.4f}.h5',
    monitor='val_dice_coef',
    verbose=1,
    save_best_only=True,
    mode='max')

callbacks = [model_checkpoint]

history = model.fit(train_epoch,
          steps_per_epoch=int(TOTAL / BATCH_SIZE),
          verbose=1,
          epochs=EPOCHS,
          shuffle=True,
          validation_data=valid_epoch,
          validation_steps=int(TOTAL_VAL / BATCH_SIZE),
          callbacks=callbacks)

train_gen.close()
valid_gen.close()

The number of total images: 1660
The number of total images: 433
Epoch 1/20
415/415 [==============================] - ETA: 0s - loss: 4.0480e-04 - dice_coef: 0.9730 - jaccard: 0.9476 - accuracy: 0.9999
Epoch 1: val_dice_coef improved from -inf to 0.78307, saving model to ./checkpoints/model_0.7831.h5
415/415 [==============================] - 358s 799ms/step - loss: 4.0480e-04 - dice_coef: 0.9730 - jaccard: 0.9476 - accuracy: 0.9999 - val_loss: 0.0291 - val_dice_coef: 0.7831 - val_jaccard: 0.6556 - val_accuracy: 0.9990
Epoch 2/20
415/415 [==============================] - ETA: 0s - loss: 4.1491e-04 - dice_coef: 0.9729 - jaccard: 0.9473 - accuracy: 0.9999
Epoch 2: val_dice_coef improved from 0.78307 to 0.78979, saving model to ./checkpoints/model_0.7898.h5
415/415 [==============================] - 293s 706ms/step - loss: 4.1491e-04 - dice_coef: 0.9729 - jaccard: 0.9473 - accuracy: 0.9999 - val_loss: 0.0252 - val_dice_coef: 0.7898 - val_jaccard: 0.6601 - val_accuracy: 0.9990
Epoch 3/20

## Evaluation

In [ ]:
from keras.models import load_model

model = load_model("./checkpoints/model_0.7938.h5",compile=False)

In [ ]:
import os
from skimage import io
import numpy as np
from generator import *

test_dataset = './data/artery/valid_artery.h5'

BATCH_SIZE = 4

print('='*32)
test_gen = DatasetReader(dataset_path=test_dataset,batch_size=BATCH_SIZE)
test_epoch = test_gen.generator()
test_images, test_labels = [], []
for i in range(test_gen.getNumberOfTotalImages()):
  image, label = test_epoch.__next__()
  test_images.append(image)
  test_labels.append(label)
print('='*32)

The number of total images: 433


In [ ]:
test_images = np.asarray(test_images)
test_labels = np.asarray(test_labels)

In [ ]:
test_images.shape

(433, 512, 512, 1)

In [ ]:
output_images = model.predict(test_images, verbose=1)

14/14 [==============================] - 40s 1s/step


In [ ]:
dir = 'predictions'
if not os.path.exists(dir):
	os.mkdir(dir)

i = 0
for image in output_images:
	image = (image[:, :, 0] * 255.).astype(np.uint8)
	ground_truth = (test_labels[i,:,:,0] * 255.).astype(np.uint8)
	origin = (test_images[i,:,:,0] *255.).astype(np.uint8)
	io.imsave(os.path.join(dir, str(i) + '_orgin.png'), origin)
	io.imsave(os.path.join(dir, str(i) + '_prediction.png'), image)
	io.imsave(os.path.join(dir, str(i) + '_groundtruth.png'), ground_truth)
	i += 1

print(f"The number of total images in test dataset are: {str(i)}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: predictions/0_prediction.png is a low contrast image
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: predictions/0_groundtruth.png is a low contrast image
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: predictions/1_prediction.png is a low contrast image
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: predictions/1_groundtruth.png is a low contrast image
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: predictions/2_prediction.png is a low contrast image
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: predictions/2_groundtruth.png is a low cont

The number of total images in test dataset are: 433


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: predictions/430_prediction.png is a low contrast image
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: predictions/430_groundtruth.png is a low contrast image
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: predictions/431_prediction.png is a low contrast image
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: predictions/431_groundtruth.png is a low contrast image
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: predictions/432_prediction.png is a low contrast image
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: predictions/432_groundtruth.png i

## Coloring

In [16]:
import cv2
import numpy as np

for i in [23,147,156,189,367]:
  img_groundtruth = cv2.imread(f"0.7938/{i}_groundtruth.png")
  img_prediction = cv2.imread(f"0.7938/{i}_prediction.png")
  blank_image = np.zeros(img_groundtruth.shape, np.uint8)

  for i in range(img_groundtruth.shape[0]):
    for j in range(len(img_groundtruth[i])):
      if (img_groundtruth[i][j][0],img_groundtruth[i][j][1],img_groundtruth[i][j][2]) == (255,255,255) and \
      (img_prediction[i][j][0],img_prediction[i][j][1],img_prediction[i][j][2]) == (254,254,254):
        (blank_image[i][j][0],blank_image[i][j][1],blank_image[i][j][2]) = (0,255,0)
      elif (img_groundtruth[i][j][0],img_groundtruth[i][j][1],img_groundtruth[i][j][2]) == (255,255,255) and \
      (img_prediction[i][j][0],img_prediction[i][j][1],img_prediction[i][j][2]) != (254,254,254):
        (blank_image[i][j][0],blank_image[i][j][1],blank_image[i][j][2]) = (0,0,255)
        # print((img_prediction[i][j][0],img_prediction[i][j][1],img_prediction[i][j][2]))
      elif (img_groundtruth[i][j][0],img_groundtruth[i][j][1],img_groundtruth[i][j][2]) != (255,255,255) and \
      (img_prediction[i][j][0],img_prediction[i][j][1],img_prediction[i][j][2]) == (254,254,254):
        (blank_image[i][j][0],blank_image[i][j][1],blank_image[i][j][2]) = (0,0,255)

  cv2.imwrite(f"0.7938/output_{i}.png",blank_image)

True